How to add Validation data in the training Process

In [ ]:
##Modified Function Header
##def train_projection_layer(dataloader, projector, val_dataloader=None, epochs=3):
##Add this block at the end of each epoch
###print(f"Epoch {epoch+1} | Loss: {avg_loss:.4f} | ANLS: {avg_anls:.4f}")


        if val_dataloader:
            projector.eval()
            val_loss = 0
            val_anls = []
            with torch.no_grad():
                for batch in val_dataloader:
                    images = batch["image"].to(device)
                    questions = batch["question"]
                    answers = batch["answer"]

                    vit_out = vit_model(pixel_values=images)
                    cls_token = vit_out.last_hidden_state[:, 0, :]
                    projected_embed = projector(cls_token).unsqueeze(1).to(torch.float16)

                    prompts = [f"Question: {q.strip()} Answer:" for q in questions]
                    full_texts = [f"{p} {a.strip()}" for p, a in zip(prompts, answers)]

                    inputs = phi_tokenizer(full_texts, return_tensors="pt", padding=True, truncation=True)
                    input_ids = inputs.input_ids.to(device)
                    attention_mask = inputs.attention_mask.to(device)

                    with phi_tokenizer.as_target_tokenizer():
                        prompt_ids = phi_tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).input_ids.to(device)

                    labels = input_ids.clone()
                    for i in range(labels.shape[0]):
                        prompt_len = (prompt_ids[i] != phi_tokenizer.pad_token_id).sum()
                        labels[i, :prompt_len] = -100

                    token_embeds = phi_model.model.embed_tokens(input_ids).to(torch.float16)
                    inputs_embeds = torch.cat([projected_embed, token_embeds], dim=1)

                    attention_mask = torch.cat([
                        torch.ones((attention_mask.shape[0], 1), dtype=torch.long).to(device),
                        attention_mask
                    ], dim=1)
                    labels = torch.cat([
                        torch.full((labels.shape[0], 1), fill_value=-100).to(device),
                        labels
                    ], dim=1)

                    outputs = phi_model(
                        inputs_embeds=inputs_embeds,
                        attention_mask=attention_mask,
                        labels=labels
                    )
                    val_loss += outputs.loss.item()

                    generated_ids = phi_model.generate(
                        inputs_embeds=inputs_embeds,
                        attention_mask=attention_mask,
                        max_new_tokens=50,
                        do_sample=False
                    )
                    decoded_preds = phi_tokenizer.batch_decode(generated_ids[:, 1:], skip_special_tokens=True)
                    decoded_preds = [pred.replace(prompt, "").strip() for pred, prompt in zip(decoded_preds, prompts)]
                    val_anls.append(compute_anls(decoded_preds, answers))

            print(f"    [Validation] Loss: {val_loss / len(val_dataloader):.4f} | ANLS: {np.mean(val_anls):.4f}")
            projector.train()
